# Простой пример калькулятора

In [23]:
%%writefile lexerinput.txt

# simple example
begin
    s = 5
    t = 1
    x = 3 + 42 * (s - t)
end

Writing lexerinput.txt


In [47]:
%%writefile calclex.py

import ply.lex as lex
import sys

reserved = {'begin': 'BEGIN', 'end': 'END', 'if' : 'IF', 'then' : 'THEN', 'else' : 'ELSE', 'while' : 'WHILE'}

tokens = ['ID', 'EQUALS', 'NUMBER', 'PLUS', 'MINUS', 'TIMES', 'DIVIDE', 'LPAREN', 'RPAREN', 'COMMENT'] + list(reserved.values())

def get_lexer():
    t_EQUALS  = r'\='
    t_PLUS    = r'\+'
    t_MINUS   = r'-'
    t_TIMES   = r'\*'
    t_DIVIDE  = r'/'
    t_LPAREN  = r'\('
    t_RPAREN  = r'\)'
    t_ignore  = ' \t'
    t_ignore_COMMENT = r'\#.*'
    
    def t_ID(token):
        r'[A-Za-z_][A-Za-z_0-9]*'
        token.type = reserved.get(token.value, 'ID')
        return token
    
    def t_NUMBER(token):
        r'\d+'
        token.value = int(token.value)
        return token
    
    def t_newline(token):
        r'\n+'
        token.lexer.lineno += len(token.value)
    
    def t_error(token):
        print(f'Illegal character: {token.value[0]}')
        token.lexer.skip(1)
        
    return lex.lex(debug=0)

def raw_input(code):
    lexer = get_lexer()
    lexer.input(code)
    for token in lexer:
        print(token)

if __name__ == '__main__':
    with open(sys.argv[1]) as src:
        raw_input(src.read())

Overwriting calclex.py


In [48]:
%%bash

python3 calclex.py lexerinput.txt

LexToken(BEGIN,'begin',3,18)
LexToken(ID,'s',4,28)
LexToken(EQUALS,'=',4,30)
LexToken(NUMBER,5,4,32)
LexToken(ID,'t',5,38)
LexToken(EQUALS,'=',5,40)
LexToken(NUMBER,1,5,42)
LexToken(ID,'x',6,48)
LexToken(EQUALS,'=',6,50)
LexToken(NUMBER,3,6,52)
LexToken(PLUS,'+',6,54)
LexToken(NUMBER,42,6,56)
LexToken(TIMES,'*',6,59)
LexToken(LPAREN,'(',6,61)
LexToken(ID,'s',6,62)
LexToken(MINUS,'-',6,64)
LexToken(ID,'t',6,66)
LexToken(RPAREN,')',6,67)
LexToken(END,'end',7,69)


In [49]:
%%writefile parserinput.txt

# simple example
3 + 42 * (5 - 1)

Overwriting parserinput.txt


In [50]:
%%writefile calcparse.py

import ply.yacc as yacc
import sys

from calclex import tokens, get_lexer

def get_parser():
    
    def p_expression_plus(p):
        'expression : expression PLUS term'
        p[0] = p[1] + p[3]
    
    def p_expression_minus(p):
        'expression : expression MINUS term'
        p[0] = p[1] - p[3]
    
    def p_expression_term(p):
        'expression : term'
        p[0] = p[1]
    
    def p_term_times(p):
        'term : term TIMES factor'
        p[0] = p[1] * p[3]
    
    def p_term_div(p):
        'term : term DIVIDE factor'
        p[0] = p[1] / p[3]
    
    def p_term_factor(p):
        'term : factor'
        p[0] = p[1]
    
    def p_factor_num(p):
        'factor : NUMBER'
        p[0] = p[1]
    
    def p_factor_expr(p):
        'factor : LPAREN expression RPAREN'
        p[0] = p[2]
    
    # Error rule for syntax errors
    def p_error(p):
        print("Syntax error in input!")
        
    return yacc.yacc(debug=1)

def parse_code(code):
    #parser = yacc.yacc(debug=1)
    parser = get_parser()
    result = parser.parse(code, lexer=get_lexer())
    print(result)

if __name__ == '__main__':
    with open(sys.argv[1]) as src:
        parse_code(src.read())

Overwriting calcparse.py


In [51]:
%%bash

python3 calcparse.py parserinput.txt

171


In [89]:
%%writefile calctree.py

import ply.yacc as yacc
import sys

from calclex import tokens, get_lexer

class Expr: pass
 
class BinOp(Expr):
    def __init__(self, left, op, right):
        self.type = "binop"
        self.left = left
        self.right = right
        self.op = op

class Number(Expr):
    def __init__(self,value):
        self.type = "number"
        self.value = value

def get_parser():
    def p_expression_binop(p):
        '''expression : expression PLUS expression
                   | expression MINUS expression
                   | expression TIMES expression
                   | expression DIVIDE expression'''
        p[0] = BinOp(p[1], p[2], p[3])
 
    def p_expression_group(p):
        'expression : LPAREN expression RPAREN'
        p[0] = p[2]
 
    def p_expression_number(p):
        'expression : NUMBER'
        p[0] = Number(p[1])
        
    return yacc.yacc(debug=1)

def print_result(op, nestingness=0):
    #new_line_mark = '|' if nestingness > 0 else ''
    new_line_mark = ''
    if isinstance(op, Number):
        print(new_line_mark + '-' * nestingness + str(op.value))
    elif isinstance(op, BinOp):
        print(new_line_mark + '-' * nestingness + op.op)
        print_result(op.left, nestingness + 1)
        print_result(op.right, nestingness + 1)

def parse_code(code):
    parser = get_parser()
    result = parser.parse(code, lexer=get_lexer())
    print_result(result)

if __name__ == '__main__':
    with open(sys.argv[1]) as src:
        parse_code(src.read())

Overwriting calctree.py


In [90]:
%%bash

python3 calctree.py parserinput.txt

+
-3
-*
--42
---
---5
---1
